In [3]:
import os

import ants
import numpy as np
import pydicom
import nibabel
import dicom2nifti
from tqdm import tqdm
from skimage.measure import find_contours
import glob
from ants import registration, from_numpy, pad_image, create_jacobian_determinant_image, plot_grid, simulate_displacement_field
from ants import create_warped_grid, plot
from PIL import Image
import scipy

In [4]:
def resize_data_volume(data, dim_list):
    """"
    Resize the data to the dim size
    Note: the ratios are not maintained because the voxels have different size and the spatial dimensions are also different
    """
    depth, height, width = data.shape
    scale = [dim_list[0] * 1.0/depth, dim_list[1] * 1.0/height, dim_list[2] * 1.0 / width]
    print(scale)
    return scipy.ndimage.interpolation.zoom(data, scale, order = 0)

In [5]:
def read_dicom_files(dicom_dir):
    try:
        dicom_files = glob.glob(os.path.join(dicom_dir, "*.*"))
        sorted_dicom_files = sorted(dicom_files)
        stacked_dicom = [pydicom.dcmread(dicom_file) for dicom_file in sorted_dicom_files]
        return stacked_dicom
    except IndexError as e:
        print(f"{e}, at path {dicom_dir}")
        return None

In [10]:
# Base path
root_path = "/media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom"
save_path_D = "/media/monib/External Disk/work2022/Base_Dataset/Resized/D"


sub_paths = [root_path + f"/{sub_dir}" for sub_dir in os.listdir(root_path)]

all_delayed_paths = []
all_portal_paths = []

for idx, sub_path in enumerate(sub_paths):

    pre_phase_path = sub_path + "/Pre/"
    artery_phase_path = sub_path + "/A/"
    portal_phase_path = sub_path + "/P/"
    delayed_phase_path = sub_path + "/D/"

    for phases_dir in (os.listdir(sub_path)):
        if "V" in phases_dir:
            portal_phase_path = portal_phase_path.split("/")
            portal_phase_path.remove("P")
            portal_phase_path = "/" + os.path.join(*portal_phase_path) + "V/"

    all_delayed_paths.append(delayed_phase_path)
    all_portal_paths.append(portal_phase_path)

patient_id_d = [d.split("/")[7] for d in all_delayed_paths]
patient_id_p = [p.split("/")[7] for p in all_portal_paths]
set_d = set(patient_id_d)
set_p = set(patient_id_p)
print(set_d.symmetric_difference(set_p))
# print(patient_id_d)

patient_id_d_sorted = sorted(patient_id_d)
patient_id_p_sorted = sorted(patient_id_p)
all_delayed_paths_sorted = sorted(all_delayed_paths)
all_portal_paths_sorted = sorted(all_portal_paths)

# for d, p in zip(all_delayed_paths_sorted, all_portal_paths_sorted):
#     print(f"p: {p}")

empty_paths = []
just_val = 0
for d_path, p_path, dir_name in tqdm(zip(all_delayed_paths_sorted, all_portal_paths_sorted, patient_id_d_sorted), desc="Progress: "):

    dicom_A_stacks = read_dicom_files(d_path)
    dicom_B_stacks = read_dicom_files(p_path)

    if len(dicom_A_stacks) == 0 or len(dicom_B_stacks) == 0:
        empty_paths.append(d_path)
        continue
    moving_shape = list(dicom_A_stacks[0].pixel_array.shape)
    moving_shape.append(len(dicom_A_stacks))

    moving_3d = np.zeros(moving_shape)

    for s in range(len(dicom_A_stacks)):
        moving_3d[:, :, s] = dicom_A_stacks[s].pixel_array

    fixed_shape = list(dicom_B_stacks[0].pixel_array.shape)
    fixed_shape.append(len(dicom_B_stacks))

    fixed_3d = np.zeros(fixed_shape)

    for j in range(len(dicom_B_stacks)):
        fixed_3d[:, :, j] = dicom_B_stacks[j].pixel_array

    print('before resize, moving 3d shape : ',moving_3d.shape )
    resized_moving = resize_data_volume(moving_3d, [256,256,moving_3d.shape[2]//2])
    print('After resize, resized_moving shape:', resized_moving.shape )

    moving_ants_img = from_numpy(moving_3d)

    save_to_path_d = os.path.join(save_path_D, dir_name + "_resized")

    os.makedirs(save_to_path_d, exist_ok=True)

    for r in range(resized_moving.shape[2]):
        dicom_slice = dicom_A_stacks[r]
        moved_slice = resized_moving[:, :, r].astype(str(dicom_slice.pixel_array.dtype))
        dicom_slice.PixelData = moved_slice.tobytes()
        dicom_slice.Rows = resized_moving.shape[0]
        dicom_slice.Columns = resized_moving.shape[1]

        save_slices_path = os.path.join(save_to_path_d, f"RS_0000{r:03}.dcm")

        dicom_slice.SeriesDescription = "[Research & Science] - Generated Data"
        dicom_slice.save_as(save_slices_path)

    just_val +=1
    if just_val == 2:
        break
print("save completed!")

set()


Progress: : 0it [00:00, ?it/s]

before resize, moving 3d shape :  (512, 512, 72)
[0.5, 0.5, 0.5]
After resize, resized_moving shape: (256, 256, 36)


Progress: : 1it [00:01,  1.41s/it]

before resize, moving 3d shape :  (512, 512, 123)
[0.5, 0.5, 0.4959349593495935]
After resize, resized_moving shape: (256, 256, 61)


Progress: : 1it [00:03,  3.82s/it]

save completed!
